In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb

In [2]:
data=pd.read_csv("../data/synthetic_dataset.csv")
data.sample(5)

,Transaction_ID,Transaction_Amount,Transaction_Date,Transaction_Time,User_Account_ID,Account_Creation_Date,Payment_Method,Billing_Location,Shipping_Location,Device_IP_Address,Session_Duration,Frequency_of_Transactions,Time_Between_Transactions,Unusual_Time_of_Transaction,Unusual_Transaction_Amounts,IP_Address_History,Fraud_Label
8610,8611,6454.55,2022-11-09 00:00:00,02:28:48,843,2022-03-07 00:00:00,Credit Card,Jabalpur,Jabalpur,236.193.28.119,245 seconds,7,452 seconds,0,2,3.155.238.7,1
7278,7279,14772.59,2022-03-16 00:00:00,12:46:11,3140,2022-05-17 00:00:00,Debit Card,Erode,Erode,115.151.45.31,629 seconds,3,127 seconds,4,5,115.151.45.31,0
3056,3057,14131.97,2022-10-14 00:00:00,20:55:44,2326,2022-12-12 00:00:00,Debit Card,Ambattur,Ambattur,195.33.228.18,570 seconds,8,1835 seconds,1,3,195.33.228.18,0
2527,2528,8729.07,2022-04-21 00:00:00,20:35:57,6653,2022-04-29 00:00:00,Debit Card,Kolkata,Kolkata,180.212.16.108,767 seconds,9,1563 seconds,1,1,180.212.16.108,0
7962,7963,12176.59,2022-03-11 00:00:00,02:49:15,9083,2022-09-25 00:00:00,Credit Card,Nashik,Nashik,4.147.254.240,408 seconds,6,1960 seconds,4,5,4.147.254.240,0


In [3]:
data1=data.drop(columns=["Transaction_ID","User_Account_ID"])

In [4]:
data1.sample(4)
# convert this into a dataframe
data1 = pd.DataFrame(data1)


In [5]:
data2=data1.copy()

In [6]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

def preprocess_data(df):
    # Convert Transaction_Date to numerical (year, month, day)
    df['Transaction_Date'] = pd.to_datetime(df['Transaction_Date'])
    df['Transaction_Year'] = df['Transaction_Date'].dt.year
    df['Transaction_Month'] = df['Transaction_Date'].dt.month
    df['Transaction_Day'] = df['Transaction_Date'].dt.day

    # Convert Transaction_Time to numerical (hour, minute, second)
    df['Transaction_Time'] = pd.to_datetime(df['Transaction_Time'])
    df['Transaction_Hour'] = df['Transaction_Time'].dt.hour
    df['Transaction_Minute'] = df['Transaction_Time'].dt.minute
    df['Transaction_Second'] = df['Transaction_Time'].dt.second

    # Convert Account_Creation_Date to numerical (year, month, day)
    df['Account_Creation_Date'] = pd.to_datetime(df['Account_Creation_Date'])
    df['Account_Creation_Year'] = df['Account_Creation_Date'].dt.year
    df['Account_Creation_Month'] = df['Account_Creation_Date'].dt.month
    df['Account_Creation_Day'] = df['Account_Creation_Date'].dt.day

    # Encoding categorical column 'Payment_Method' using Label Encoding
    label_encoder = LabelEncoder()
    df['Payment_Method'] = label_encoder.fit_transform(df['Payment_Method'])
    df['Billing_Location']=label_encoder.fit_transform(df['Billing_Location'])
    df['Shipping_Location']=label_encoder.fit_transform(df['Shipping_Location'])
    # Convert Session_Duration and Time_Between_Transactions to numerical (remove 'seconds' suffix)
    df['Session_Duration'] = df['Session_Duration'].apply(lambda x: int(x.split()[0]))
    df['Time_Between_Transactions'] = df['Time_Between_Transactions'].apply(lambda x: int(x.split()[0]))
    
    

    # Convert IP_Address_History and Device_IP_Address to numerical (last part)
    df['IP_Address_History'] = df['IP_Address_History'].apply(lambda x: int(x.split('.')[-1]))
    df['Device_IP_Address'] = df['Device_IP_Address'].apply(lambda x: int(x.split('.')[-1]))

    # Dropping original non-useful columns after preprocessing
    columns_to_drop = ['Transaction_Date', 'Transaction_Time', 'Account_Creation_Date',]
    df.drop(columns=columns_to_drop, inplace=True)

    return df

# Usage example:
preprocess_data(data1)


C:\Users\Yash\AppData\Local\Temp\ipykernel_10136\3387447770.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Transaction_Time'] = pd.to_datetime(df['Transaction_Time'])


,Transaction_Amount,Payment_Method,Billing_Location,Shipping_Location,Device_IP_Address,Session_Duration,Frequency_of_Transactions,Time_Between_Transactions,Unusual_Time_of_Transaction,Unusual_Transaction_Amounts,...,Fraud_Label,Transaction_Year,Transaction_Month,Transaction_Day,Transaction_Hour,Transaction_Minute,Transaction_Second,Account_Creation_Year,Account_Creation_Month,Account_Creation_Day
0,19122.08,0,31,31,225,454,4,466,0,1,...,0,2022,12,18,14,32,43,2022,5,4
1,2090.68,2,0,0,8,806,2,203,3,4,...,0,2022,2,24,7,58,38,2022,11,16
2,9247.22,0,30,30,158,462,3,1422,2,3,...,0,2022,8,9,12,59,59,2022,12,29
3,7686.80,1,68,68,60,363,1,1895,1,3,...,0,2022,12,22,0,20,42,2022,2,14
4,3693.87,1,95,69,188,263,8,1281,4,0,...,0,2022,2,21,19,20,27,2022,8,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,270.93,1,15,95,72,793,10,1082,2,4,...,0,2022,10,15,23,45,15,2022,8,9
9996,17618.49,2,70,70,94,780,1,883,1,2,...,0,2022,1,6,16,13,36,2022,6,1
9997,14286.99,0,100,100,184,275,1,106,4,4,...,0,2022,9,8,13,4,17,2022,6,11
9998,3036.76,1,1,71,6,602,4,1997,2,2,...,0,2022,12,17,0,42,10,2022,1,22


In [7]:
data1.dtypes

Transaction_Amount             float64
Payment_Method                   int32
Billing_Location                 int32
Shipping_Location                int32
Device_IP_Address                int64
Session_Duration                 int64
Frequency_of_Transactions        int64
Time_Between_Transactions        int64
Unusual_Time_of_Transaction      int64
Unusual_Transaction_Amounts      int64
IP_Address_History               int64
Fraud_Label                      int64
Transaction_Year                 int32
Transaction_Month                int32
Transaction_Day                  int32
Transaction_Hour                 int32
Transaction_Minute               int32
Transaction_Second               int32
Account_Creation_Year            int32
Account_Creation_Month           int32
Account_Creation_Day             int32
dtype: object

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

# Assuming 'data' contains the provided dataset after preprocessing

# Splitting into features (X) and target variable (y)
X = data1.drop(columns=['Fraud_Label'])
y = data1['Fraud_Label']

# Splitting the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize XGBoost model
model = XGBClassifier(random_state=42)

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate model performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Display classification report
print(classification_report(y_test, y_pred))


Accuracy: 0.95
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      1586
           1       1.00      0.78      0.88       414

    accuracy                           0.95      2000
   macro avg       0.97      0.89      0.92      2000
weighted avg       0.96      0.95      0.95      2000



In [10]:
import pandas as pd

# Sample input data
sample_input = {
    'Transaction_Amount': 123530.6,
    'Transaction_Date': '2023-06-08',
    'Transaction_Time': '15:00:00',
    'Account_Creation_Date': '2022-08-01',
    'Payment_Method': 'UPI',
    'Billing_Location': 'Mumbai',
    'Shipping_Location': 'Mumbai',
    'Device_IP_Address': '203.45.67.113',
    'Session_Duration': '720 seconds',
    'Frequency_of_Transactions': 9,
    'Time_Between_Transactions': '150 seconds',
    'Unusual_Time_of_Transaction': 1,
    'Unusual_Transaction_Amounts': 0,
    'IP_Address_History': '203.45.67.113'
}

# Convert the sample input data to a DataFrame
sample_df = pd.DataFrame([sample_input])

# Preprocess the sample input data (similar to preprocessing done for training data)
# Assuming preprocess_data() is the function used for preprocessing the training data
sample_df_processed = preprocess_data(sample_df)

# Display the preprocessed sample input data
print(sample_df_processed)


   Transaction_Amount  Payment_Method  Billing_Location  Shipping_Location  \
0            123530.6               0                 0                  0   

   Device_IP_Address  Session_Duration  Frequency_of_Transactions  \
0                113               720                          9   

   Time_Between_Transactions  Unusual_Time_of_Transaction  \
0                        150                            1   

   Unusual_Transaction_Amounts  IP_Address_History  Transaction_Year  \
0                            0                 113              2023   

   Transaction_Month  Transaction_Day  Transaction_Hour  Transaction_Minute  \
0                  6                8                15                   0   

   Transaction_Second  Account_Creation_Year  Account_Creation_Month  \
0                   0                   2022                       8   

   Account_Creation_Day  
0                     1  


In [11]:
# Assuming 'model' is the trained XGBoost model from the previous example

# Make predictions on the preprocessed sample input data
fraud_prediction = model.predict(sample_df_processed)

# Display the predicted fraud label
print(f"Predicted Fraud Label: {fraud_prediction[0]}")


Predicted Fraud Label: 1


In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier

# Initialize classifiers
rf_classifier = RandomForestClassifier(random_state=42)
xgb_classifier = XGBClassifier(random_state=42)
nn_classifier = MLPClassifier(random_state=42)

# Create an ensemble using VotingClassifier
ensemble_classifier = VotingClassifier(
    estimators=[
        ('random_forest', rf_classifier),
        ('xgboost', xgb_classifier),
        ('neural_network', nn_classifier)
    ],
    voting='soft'  # Use soft voting for probabilities averaging
)

# Training the ensemble
ensemble_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred_ensemble = ensemble_classifier.predict(X_test)

# Evaluate the ensemble model performance
accuracy_ensemble = accuracy_score(y_test, y_pred_ensemble)
print(f"Ensemble Accuracy: {accuracy_ensemble:.2f}")

# Display classification report for the ensemble
print(classification_report(y_test, y_pred_ensemble))

# Use the ensemble to predict fraud label for the sample input
fraud_prediction_ensemble = ensemble_classifier.predict(sample_df_processed)
print(f"Ensemble Predicted Fraud Label for Sample Input: {fraud_prediction_ensemble[0]}")


Ensemble Accuracy: 0.88
              precision    recall  f1-score   support

           0       0.87      1.00      0.93      1586
           1       1.00      0.42      0.59       414

    accuracy                           0.88      2000
   macro avg       0.93      0.71      0.76      2000
weighted avg       0.90      0.88      0.86      2000

Ensemble Predicted Fraud Label for Sample Input: 1


In [13]:
# Sample input data
sample_input = {
    'Transaction_Amount': 434324,
    'Transaction_Date': '2023-06-08',
    'Transaction_Time': '15:00:00',
    'Account_Creation_Date': '2022-08-01',
    'Payment_Method': 'UPI',
    'Billing_Location': 'Mumbai',
    'Shipping_Location': 'Mumbai',
    'Device_IP_Address': '203.45.67.113',
    'Session_Duration': '7230 seconds',
    'Frequency_of_Transactions': 9,
    'Time_Between_Transactions': '150 seconds',
    'Unusual_Time_of_Transaction': 1,
    'Unusual_Transaction_Amounts': 0,
    'IP_Address_History': '203.435.67.113'
}



# Convert the sample input data to a DataFrame
sample_df = pd.DataFrame([sample_input])

# Preprocess the sample input data (similar to preprocessing done for training data)
# Assuming preprocess_data() is the function used for preprocessing the training data
sample_df_processed = preprocess_data(sample_df)

# Display the preprocessed sample input data
print(sample_df_processed)

   Transaction_Amount  Payment_Method  Billing_Location  Shipping_Location  \
0              434324               0                 0                  0   

   Device_IP_Address  Session_Duration  Frequency_of_Transactions  \
0                113              7230                          9   

   Time_Between_Transactions  Unusual_Time_of_Transaction  \
0                        150                            1   

   Unusual_Transaction_Amounts  IP_Address_History  Transaction_Year  \
0                            0                 113              2023   

   Transaction_Month  Transaction_Day  Transaction_Hour  Transaction_Minute  \
0                  6                8                15                   0   

   Transaction_Second  Account_Creation_Year  Account_Creation_Month  \
0                   0                   2022                       8   

   Account_Creation_Day  
0                     1  


In [14]:
def predict_fraud_label(sample_df_processed, model):
    fraud_prediction = model.predict(sample_df_processed)
    return fraud_prediction[0]

# Assuming 'ensemble_classifier' is the trained ensemble model
predicted_label = predict_fraud_label(sample_df_processed, ensemble_classifier)
print(f"Predicted Fraud Label for the Sample Input: {predicted_label}")

Predicted Fraud Label for the Sample Input: 1
